In [2]:
import logging
from importlib import reload
reload(logging)
logger = logging.getLogger(__name__)

In [3]:
import ast
import pandas as pd
import time
import random
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from pprint import pprint
from pathlib import Path
import numpy as np
from llm_assessor import (
    run_prompt_chain,
    extract_marks_from_graded_answer,
)

load_dotenv(override=True)

ERROR:root:Failed to update token costs. Using static costs.
/Users/amiralkateb/.virtualenvs/structure_markschemes/lib/python3.10/site-packages/tokencost/constants.py:61: RuntimeWarning: coroutine 'update_token_costs' was never awaited
  TOKEN_COSTS = TOKEN_COSTS_STATIC


True

In [5]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

'/Users/amiralkateb/Documents/GitRepos/audenshaw_exam_validation'

In [42]:
# helper functions
def findLevel(num, levels_dict):
    for level in levels_dict.keys():
        if min(levels_dict.get(level)) <= num <= max(levels_dict.get(level)):
            return int(level)
    return None
def compute_metric(llm_mark, teacher_mark, level_structure):
    awarded_level = findLevel(teacher_mark,level_structure )
    llm_level = findLevel(llm_mark, level_structure)
    print('llm mark', llm_mark)
    print('awarded_level', awarded_level)
    print('llm level', llm_level)
    dif_level = abs(awarded_level - llm_level) + 1
    metric = dif_level * abs(teacher_mark- llm_mark)
    return metric

In [34]:
# Load Directories containing Prompt Templates and Student Answers
prompt_dir = Path("prompt_templates")

data_dir = Path(f"{workdir}/validation_results/processed_data")

In [35]:
# Load LLM
llm = ChatOpenAI(
    model_name="gpt-4o", 
    temperature=0.00, 
    max_tokens=800,
    )

In [36]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt="%Y-%m-%d %H:%M:%S",
    )


# Load in Student Answers

In [38]:
# Load in Data
completed_paper_df = pd.read_csv(data_dir / "student_answers_augmented.csv")
#completed_paper_df = completed_paper_df[(completed_paper_df.subject_id == "edexcel_business_studies") & (completed_paper_df.student_id == 5095)]
student_answers_records = completed_paper_df.to_dict(orient="records")

# Read in Prompt Templates

In [39]:
# Load all prompt templates
prompts = {
    "grade_answer": 
    {
        "aqa_history": {
            "hs_analyse": load_prompt(prompt_dir/ "aqa_history" / "analyse_prompt" / "grade_answer_aqa_history_analyse_prompt.json"),
            "hs_explain": load_prompt(prompt_dir/ "aqa_history" / "explain_prompt" / "grade_answer_aqa_history_explain_prompt.json"),
            "hs_judgement": load_prompt(prompt_dir/ "aqa_history" / "judgement_prompt" / "grade_answer_aqa_history_judgement_prompt.json"),
            "hs_spag": load_prompt(prompt_dir/ "aqa_history" / "spag_prompt" / "grade_answer_aqa_history_spag_prompt.json")
        },
        "edexcel_business_studies": {
            "bs_analyse": load_prompt(prompt_dir/ "edexcel_business_studies" / "analyse_prompt" / "grade_answer_edexcel_business_studies_analyse_prompt.json"),
            "bs_discuss": load_prompt(prompt_dir/ "edexcel_business_studies" / "discuss_prompt" / "grade_answer_edexcel_business_studies_discuss_prompt.json"),
            "bs_evaluate": load_prompt(prompt_dir/ "edexcel_business_studies" / "evaluate_prompt" / "grade_answer_edexcel_business_studies_evaluate_prompt.json"),
            "bs_explain": load_prompt(prompt_dir/ "edexcel_business_studies" / "explain_prompt" / "grade_answer_edexcel_business_studies_explain_prompt.json"),
            "bs_identify": load_prompt(prompt_dir/ "edexcel_business_studies" / "identify_prompt" / "grade_answer_edexcel_business_studies_identify_prompt.json"),
            "bs_justify": load_prompt(prompt_dir/ "edexcel_business_studies" / "justify_prompt" / "grade_answer_edexcel_business_studies_justify_prompt.json"),
            "bs_outline": load_prompt(prompt_dir/ "edexcel_business_studies" / "outline_prompt" / "grade_answer_edexcel_business_studies_outline_prompt.json"),
            "bs_state": load_prompt(prompt_dir/ "edexcel_business_studies" / "state_prompt" / "grade_answer_edexcel_business_studies_state_prompt.json"),
            "bs_mcq": load_prompt(prompt_dir/ "edexcel_business_studies" / "mcq_prompt" / "grade_answer_edexcel_business_studies_mcq_prompt.json"),
            "bs_calculate": load_prompt(prompt_dir/ "edexcel_business_studies" / "calculate_prompt" / "grade_answer_edexcel_business_studies_calculate_prompt.json")
        }
    },
    "extract_marks": load_prompt(prompt_dir / "extract_mark_count" / "extract_mark_count_prompt.json"),
    }

# Grade Answers for Each Student

## Pass 1: Grade all Student Answers with Few Shot Prompting

In [49]:
# Pass 1: Grade All Answers
valid_types = [
    'bs_discuss', 'bs_analyse', 'bs_justify', 'bs_evaluate', 
    'hs_explain', 'hs_analyse', 'hs_judgement', 'hs_spag'
]
#student_answers_records['level_structure'].where(student_answers_records['level_structure'] is not None and student_answers_records['question_type'] not in valid_types)
#out = student_answers_records[student_answers_records['question_type'] not in valid_types & student_answers_records['level_structure'] is not None ]['level_structure']

for _, student_answer in enumerate(student_answers_records):
    
    student_answer["start_time"] = time.time()
    prompt_template = prompts["grade_answer"].get(student_answer["subject_id"]).get(student_answer["question_type"])
    
    if (student_answer["answer_text"] is np.nan or len(student_answer["answer_text"]) == 0):
        
        student_answer["llm_graded_answer"] = "No Answer is provided. Therefore 0 marks are awarded for this answer."   
        student_answer["llm_graded_answer_token_costing"] = 0
        student_answer["llm_awarded_marks"] = 0
        student_answer["llm_awarded_marks_token_costing"] = 0
    
    else:
        try:
            # Grade Answer
            logging.info(f"Starting Grading Answer for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

            input_args = {
                "question": student_answer["question_text"], 
                "answer": student_answer["answer_text"], 
                "mark_scheme": student_answer["mark_scheme_text"], 
                "context": student_answer["context"]
                }
            prompt_template_input_args = dict((k, input_args[k]) for k in prompt_template.input_variables if k in input_args)

            graded_answer_response = run_prompt_chain(
                prompt_template=prompt_template.template,
                llm=llm,
                burn_in_runs=1,
                **prompt_template_input_args
                )
            
            student_answer["llm_graded_answer"] = graded_answer_response["prompt_chain_response"]
            student_answer["llm_graded_answer_token_costing"] = graded_answer_response["prompt_chain_token_costing"]
            
            logging.info(f"Completed Grading Answer for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")
            
            # Extract Marks
            logging.info(f"Starting Extracting Marks for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")
            
            extract_marks_response = run_prompt_chain(
                prompt_template="Extract the number of marks awarded to the student answer. Read the answer carefully and extract the final number of marks to be awarded to the student. Report the number of marks only. Answer: {answer}",
                llm=ChatOpenAI(model_name="gpt-4o", temperature=0.0, max_tokens=500),
                burn_in_runs=1,
                answer=student_answer["llm_graded_answer"]
                )
            marks = extract_marks_response["prompt_chain_response"]
            marking_token_cost = extract_marks_response["prompt_chain_token_costing"]

            student_answer["llm_awarded_marks"] = int(marks)
            student_answer["llm_awarded_marks_token_costing"] = marking_token_cost
            
            logging.info(f"Completed Extracting Marks for Student ID: {student_answer['student_id']} and Question ID: {student_answer['question_id']}")

        except:
            logging.info(f"Question type: {student_answer['question_type']} is not supported. Assigning NA and skipping to next question.")

            student_answer["llm_graded_answer"] = "NA"   
            student_answer["llm_graded_answer_token_costing"] = 0
            student_answer["llm_awarded_marks"] = np.nan
            student_answer["llm_awarded_marks_token_costing"] = 0

    # Compute hitrate
    min_mark = min( [student_answer.get("awarded_marks")] )
    max_mark = max( [student_answer.get("awarded_marks")] )

    if (student_answer["awarded_marks"] == 0 and student_answer["answer_text"] is np.nan):
        student_answer["llm_mark_hitrate"] = np.nan
    else:
        student_answer["llm_mark_hitrate"]=min_mark<=student_answer["llm_awarded_marks"]<=max_mark
    valid_types = [
    'bs_discuss', 'bs_analyse', 'bs_justify', 'bs_evaluate', 
    'hs_explain', 'hs_analyse', 'hs_judgement', 'hs_spag'
]
    if student_answer['level_structure'] is not None and student_answer['question_type'] in valid_types:
        
        levels_dict = ast.literal_eval(student_answer['level_structure'])
        
        max_error = max(compute_metric(0, student_answer['awarded_marks'], levels_dict), compute_metric(max(max(lst) for lst in levels_dict.values()), student_answer['awarded_marks'], levels_dict) )
        print('max error',max_error)
        try:
            metric = compute_metric(llm_mark=student_answer['llm_awarded_marks'], teacher_mark=student_answer['awarded_marks'], level_structure=levels_dict)
            print('metric:', metric, student_answer['awarded_marks'], student_answer['llm_awarded_marks'])
            dif_level = abs(findLevel(student_answer['llm_awarded_marks'], levels_dict) - findLevel(student_answer['awarded_marks'], levels_dict))
            student_answer['same_level_hitrate'] = int(dif_level <=1)
            student_answer['same_level_hitrate_pm1'] = int(dif_level <=1 and abs(student_answer["awarded_marks"] - student_answer["llm_awarded_marks"]) <=1)
        except:
            metric = max_error
    

        scaled_error = metric/max_error
        student_answer['scaled_error_metric'] = scaled_error
        print('scaled error', scaled_error)
        #compute same level hitrate
        



    
    

    student_answer["end_time"] = time.time()
    student_answer["elapsed_time_in_seconds"] = student_answer["end_time"] - student_answer["start_time"]

    # Randomly sleep for seconds to avoid API throttling. Between 1-3 seconds
    seconds_to_sleep = random.sample([1,2,3], 1)[0]

    logging.info(f"Sleeping for {seconds_to_sleep} seconds to avoid API Throttling.")
    time.sleep(seconds_to_sleep)    

2024-08-15 18:34:20 - INFO - Starting Grading Answer for Student ID: 5000 and Question ID: 1.0
2024-08-15 18:34:20 - INFO - Construct LLM Chain
2024-08-15 18:34:20 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-08-15 18:34:23 - INFO - Completed Grading Answer for Student ID: 5000 and Question ID: 1.0
2024-08-15 18:34:23 - INFO - Starting Extracting Marks for Student ID: 5000 and Question ID: 1.0
2024-08-15 18:34:23 - INFO - Construct LLM Chain
2024-08-15 18:34:23 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-08-15 18:34:24 - INFO - Completed Extracting Marks for Student ID: 5000 and Question ID: 1.0
2024-08-15 18:34:24 - INFO - Sleeping for 3 seconds to avoid API Throttling.


llm mark 0
awarded_level 1
llm level 0
llm mark 4
awarded_level 1
llm level 2
max error 6
llm mark 2
awarded_level 1
llm level 1
metric: 1 1 2
scaled error 0.16666666666666666


2024-08-15 18:34:27 - INFO - Starting Grading Answer for Student ID: 5000 and Question ID: 2.0
2024-08-15 18:34:27 - INFO - Construct LLM Chain
2024-08-15 18:34:27 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-08-15 18:34:31 - INFO - Completed Grading Answer for Student ID: 5000 and Question ID: 2.0
2024-08-15 18:34:31 - INFO - Starting Extracting Marks for Student ID: 5000 and Question ID: 2.0
2024-08-15 18:34:31 - INFO - Construct LLM Chain
2024-08-15 18:34:31 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-08-15 18:34:32 - INFO - Completed Extracting Marks for Student ID: 5000 and Question ID: 2.0
2024-08-15 18:34:32 - INFO - Sleeping for 3 seconds to avoid API Throttling.


llm mark 0
awarded_level 2
llm level 0
llm mark 12
awarded_level 2
llm level 4
max error 24
llm mark 5
awarded_level 2
llm level 2
metric: 1 4 5
scaled error 0.041666666666666664


2024-08-15 18:34:35 - INFO - Starting Grading Answer for Student ID: 5000 and Question ID: 3.0
2024-08-15 18:34:35 - INFO - Construct LLM Chain
2024-08-15 18:34:35 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-08-15 18:34:40 - INFO - Completed Grading Answer for Student ID: 5000 and Question ID: 3.0
2024-08-15 18:34:40 - INFO - Starting Extracting Marks for Student ID: 5000 and Question ID: 3.0
2024-08-15 18:34:40 - INFO - Construct LLM Chain
2024-08-15 18:34:40 - INFO - Run LLM Chain with 1 Burn in Run(s)
2024-08-15 18:34:41 - INFO - Completed Extracting Marks for Student ID: 5000 and Question ID: 3.0
2024-08-15 18:34:41 - INFO - Sleeping for 2 seconds to avoid API Throttling.


...

llm mark 0
awarded_level 0
llm level 0
llm mark 9
awarded_level 0
llm level 3
max error 36
llm mark 0
awarded_level 0
llm level 0
metric: 0 0 0
scaled error 0.0


2024-08-15 18:52:26 - INFO - Sleeping for 2 seconds to avoid API Throttling.
2024-08-15 18:52:28 - INFO - Sleeping for 1 seconds to avoid API Throttling.
2024-08-15 18:52:29 - INFO - Sleeping for 3 seconds to avoid API Throttling.
2024-08-15 18:52:32 - INFO - Sleeping for 2 seconds to avoid API Throttling.


llm mark 0
awarded_level 0
llm level 0
llm mark 9
awarded_level 0
llm level 3
max error 36
llm mark 0
awarded_level 0
llm level 0
metric: 0 0 0
scaled error 0.0


2024-08-15 18:52:34 - INFO - Sleeping for 1 seconds to avoid API Throttling.


llm mark 0
awarded_level 0
llm level 0
llm mark 12
awarded_level 0
llm level 3
max error 48
llm mark 0
awarded_level 0
llm level 0
metric: 0 0 0
scaled error 0.0


In [ ]:
llmgraded = [0,1,2,3,4,5,6,7,8,9,10,11,12]
levels = {0: [0], 1: [1,2,3], 2: [4,5,6], 3: [7,8,9], 4: [10, 11, 12]}
real = 4

metric = compute_metric(llm_mark=12, teacher_mark=real, level_structure=levels)
print(metric)
max_error = max(compute_metric(0, real, levels), compute_metric(max(max(lst) for lst in levels_dict.values()), real, levels) )
print(max_error)

scaled_error = metric/max_error
print('scaled error', scaled_error)





24
24
scaled error 1.0


In [51]:
# Combine Student Answers and save as a DataFrame
completed_paper_df = pd.DataFrame(student_answers_records)
ordered_cols = [
    'subject_id', 'question_id', 'question_type', 'student_id',
    'question_text', 'mark_scheme_text', 'context', 'answer_text', 
    'llm_graded_answer', 'awarded_marks', 'llm_awarded_marks', 'total_marks',
    'llm_mark_hitrate' ,'scaled_error_metric', 'same_level_hitrate','same_level_hitrate_pm1', 'answer_id', 'linked_answer_id', 'topic_id', 
    'answer_scanned_image', 'elapsed_time_in_seconds', 'llm_graded_answer_token_costing', 'llm_awarded_marks_token_costing'
    ]

completed_paper_df = completed_paper_df[ ordered_cols]

In [ ]:
# student_answer = [i for i in student_answers_records if i["question_id"] == 3.4][0]
# student_answer

In [ ]:
# print("Mark Scheme:\n", student_answer["mark_scheme_text"])
# print("-"*40)
# print("Student Answer:\n", student_answer["answer_text"])
# print("-"*40)
# print("Graded Answer:\n", student_answer["llm_graded_answer"])
# print("-"*40)
# print("LLM Marks:\n", student_answer["llm_awarded_marks"])
# print("-"*40)
# print("Actual Marks:\n", student_answer["awarded_marks"])

In [52]:
# Save graded student answers
completed_paper_df.to_csv(data_dir / "student_answers_llm_graded.csv", index=False)